In [1]:
# Test the prompts
import json 
from codesim.prompt import StraightLine as SL
%load_ext autoreload
%autoreload 2

# Get the data
data = json.load(open("./data/StraightLine/n_ops-20_n_vars-3_n_instances-2_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
var_name = list(lab_syn.keys())[0]  # can sample here
gt = lab_syn[var_name]
question = SL.questions_syn[0].format(varname=var_name)
prompt_syn = SL.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)

print("="*30)
# Nat
agent_name = list(lab_nat.keys())[0]  # can sample here
object_name = list(lab_nat[agent_name].keys())[0]  # same...
gt = lab_nat[agent_name][object_name]
question = SL.questions_nat[0].format(varname=object_name, agentname=agent_name)
prompt_nat = SL.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

#### Critical Path

In [5]:
# Test the prompts
import json 
import random
from codesim.prompt import CriticalPath as CP
%reload_ext autoreload
%autoreload 2

# Get the data
data = json.load(open("./data/CriticalPath/n_ops-20_n_vars-4_len_critical_path-10_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
idx_label = random.sample(list(lab_syn.keys()), 1)var_name = list(lab_syn.keys())[0idx_label  # can sample here
gt = lab_syn[var_name]
question = CP.questions_syn[0].format(varname=var_name)
prompt_syn = CP.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)

print("="*30)
# Nat
agent_name = list(lab_nat.keys())[0]  # can sample here
gt = lab_nat[agent_name]
question = CP.questions_nat[0].format(agentname=agent_name)
prompt_nat = CP.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)


SyntaxError: invalid decimal literal (2126331429.py, line 13)

#### Parallel Paths

In [37]:
# Test the prompts
import json 
from codesim.prompt import ParallelPaths as PP

# Get the data
data = json.load(open("./data/ParallelPaths/n_ops-20_n_vars-3_n_instances-2_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
var_names = list(lab_syn.keys())  # can sample here
# gt = lab_syn[var_name]
question = PP.questions_syn[0].format(varnames=", ".join(var_names))
prompt_syn = PP.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)

print("="*30)
# Nat
# object_name = list(lab_nat[agent_name].keys())  # same...
# gt = lab_nat[agent_name][object_name]
question = PP.questions_nat[0]
prompt_nat = PP.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

#### Sort

In [ ]:
# Test the prompts
import json 
from codesim.prompt import Sort

# Get the data
data = json.load(open("./data/ParallelPaths/n_vars-10_ascending-True_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
position = lab_syn["position"]
ascending = lab_syn["ascending"]
question = Sort.questions_syn[0].format(k=position, ascending= "biggest" if ascending else "smallest")
prompt_syn = Sort.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)
# Nat
# object_name = list(lab_nat[agent_name].keys())  # same...
# gt = lab_nat[agent_name][object_name]
question = Sort.questions_nat[0]
prompt_nat = Sort.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

#### Loops

#### Boxes

### 3. Test exec-time SambaNova

In [9]:
import os
import openai
import json
from tqdm import tqdm
from time import sleep
from codesim.prompt import StraightLine as SL

client = openai.OpenAI(
    api_key="380072e2-1a5e-4d75-b7a7-d657a457c416",
    base_url="https://api.sambanova.ai/v1",
)

def call(x:str) -> str:
    response = client.chat.completions.create(
        model='Meta-Llama-3.1-405B-Instruct',
        messages=[{"role":"system","content":"You are a helpful assistant"},{"role":"user","content":x}],
        temperature =  0.1,
        top_p = 0.1
    )

    return response.choices[0].message.content

# Get the data
data = json.load(open("./data/StraightLine/n_ops-20_n_vars-3_n_instances-2_batch-1.json"))

for idx, d in tqdm(enumerate(data)):
    p_syn, p_nat, lab_syn, lab_nat = d.values()
    var_name = list(lab_syn.keys())[0]  # can sample here
    gt = lab_syn[var_name]
    question = SL.questions_syn[0].format(varname=var_name)
    prompt_syn = SL.user_cot.format(prefix="Here's some code:", 
                                    problem=p_syn, 
                                    question=question)
    responses = {}
    try:
        responses[idx] = { "prompt": prompt_syn,
                           "response": call(prompt_syn),
                           "gt": gt
                        }
    except:
        sleep(5)
        
json.dump(responses, open("responses.json", "w"), indent=4)

### 1. Retrieve and save data

In [1]:
from codesim.problem import StraightLine 

gen = StraightLine(n_ops=20, 
                   n_vars=3,
                   n_instances=2)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()


In [2]:
from codesim.problem import CriticalPath 

gen = CriticalPath(n_ops=20, 
                   n_vars=4,
                   len_critical_path=10)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [3]:
from codesim.problem import ParallelPaths 

gen = ParallelPaths(n_ops=20, 
                    n_vars=3,
                    n_instances=2)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [4]:
from codesim.problem import Loops 

gen = Loops(n_loops=5, 
            n_noisy_loops=2,
            min_loop_length=3,
            max_loop_length=5)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [5]:
from codesim.problem import Sort 

gen = Sort(n_vars=10,
           ascending=True)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [ ]:
import sys
from codesim.kim_schuster import real_main
%load_ext autoreload
%autoreload 2
# simulate CLI run
sys.argv = ["codesim/kim_schuster.py", 
            "--max_items_per_box", "3",
            "--num_samples", "30"]

real_main()

### 2. Generate prompts

#### Straight Line